In [11]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
from itertools import product
from difflib import SequenceMatcher
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler


import re
import json
from pathlib import Path

import pandas as pd
import numpy as np

In [12]:
tpch_dir = Path("../data/tpchv2/")
tpcc_dir = Path("../data/tpccv2/")

### インスタンスが独立に動作していると考え，インスタンスごと(=hostごと)にDataFrameを作成
- apjのdfには"read", geodataのdfには"write"ラベルを貼る
- tpchは後で付け足す感じで．(readのラベルを貼る)

In [16]:
df_lst = []

# tpch & tpcc (pseudo data)
for data_dir in [tpcc_dir, tpch_dir]:
    data_dict = {}
    json_paths = data_dir.glob("*.json")
    for path in json_paths:
        with open(path, "r") as f:
            metric = path.name[:-24]
            results = json.load(f)["results"]
            metric_set = set()
            for src_data in results:
                for data in src_data["data"]:
                    metric_set.add(data["metric"])
            for src_data in results:
                if src_data["source"][:10] != "summarizer":
                    for data in src_data["data"]:
                        # jsonのキーは順番保証されていないので念の為ソート
                        tags = [data["tags"][key] for key in sorted(data["tags"].keys()) if key != "origin"]
                        key_name = metric
                        # metricフィールドが全て同じならキー名に含めない
                        if len(metric_set) > 1:
                            # metricフィールドの冗長な名前を簡潔化
                            metric_attr = re.findall('^.*\.(.*)$', data['metric'])[0]
                            key_name += f"-{metric_attr}"
                        if len(tags) > 0:
                            key_name += f"-{'-'.join(tags)}"
                        data_dict[key_name] = data["NumericType"]
    df = pd.DataFrame(data_dict, dtype="float32")
    df.dropna(inplace=True)
    # query数が0近くから急激に上昇した直後 or 急激に0近くまで下降した直前のデータを取り除く
    if data_dir == tpcc_dir:
        query_nums = df.filter(like="query-").sum(axis=1)
        rm_indices = set(query_nums[query_nums < 500].index.to_list())
        rm_indices = rm_indices | {idx + 1 for idx in rm_indices if idx < len(df)} | {idx - 1 for idx in rm_indices if idx > 1} | {1, len(df)}
    else:
        cpu_usage = df.filter(like="cpu-usage_system").sum(axis=1)
        rm_indices = set(cpu_usage[cpu_usage < 8].index.to_list())
        rm_indices = rm_indices | {idx + 1 for idx in rm_indices if idx < len(df)} | {idx - 1 for idx in rm_indices if idx > 1} | {1, len(df)}
    df.drop(rm_indices, axis=0, inplace=True)
    label = "read" if data_dir == tpch_dir else "write"
    df["label"] = [label] * len(df) # ラベルを追加
    df_lst.append(df)

In [18]:
# ワークロード間で重複していないメトリックを探索
duplicated_cols = set(df_lst[0].columns)
for tmp_df in df_lst:
    duplicated_cols &= set(tmp_df.columns)

non_duplicated_cols = set()
for tmp_df in df_lst:
    non_duplicated_cols |= set(tmp_df.columns) ^ duplicated_cols

print("non duplicated cols:")
print(non_duplicated_cols)

# 重複したメトリックを削除
if len(non_duplicated_cols) > 0:
    for tmp_df in df_lst:
        tmp_df.drop(non_duplicated_cols, axis=1, errors='ignore', inplace=True)

non duplicated cols:
{'query_response_time', 'no_index_used_total'}


### 閾値を徐々に下げて削除していく
これにより相関が非常に高いものを優先的に削除できる

- 特徴量を選択する際は全てのwrite, read intensiveなデータを一緒くたにして相関をみる

In [34]:
from pprint import pprint

sns.set(font_scale=2)
# df = pd.concat(df_lst[:-1])
df = pd.concat(df_lst)
df = df.loc[:, df.nunique() != 1] # 値が一定のメトリックを除く
df_selected = df.drop(["label"], axis=1, inplace=False)

labels = df["label"]

del_lim = 35
del_num = 0
ths = [0.95, 0.9, 0.85, 0.8, 0.75, 0.7]
del_metrics_all = []

for th in ths:
    df_corr = df_selected.corr()
    corr_mat = df_corr.to_numpy()
    cols = df_corr.columns

    # 相関が th 以上 or -th 以下のメトリックを取り出す
    high_corrs_dict = {k: set() for k in cols}
    for i, j in zip(*np.where((corr_mat >= th) | (corr_mat <= -th))):
        if i < j:
            # queryはworkloadを最もよく表しているので，消さないようにする
            if cols[i][:6] != "query-":
                high_corrs_dict[cols[i]].add(cols[j])
            if cols[j][:6] != "query-":
                high_corrs_dict[cols[j]].add(cols[i])
    del_metrics = []
    while del_num < del_lim:
        # 相関が高いメトリック間の関係数をメトリック別に列挙
        # （メトリックごとの関係数を相関係数の和で代用してもいい）
        del_metric = max(high_corrs_dict.items(), key=lambda item: len(item[1]))[0]
        if len(high_corrs_dict[del_metric]) == 0:
            break
        # keyを削除
        high_corrs_dict.pop(del_metric, None)
        # value(=set)の要素を削除
        for k, v_set in high_corrs_dict.items():
            if del_metric in v_set:
                v_set.discard(del_metric)
        del_metrics.append(del_metric)
        del_num += 1
    del_metrics_all += del_metrics
    df_selected.drop(del_metrics, axis=1, inplace=True)
pprint(del_metrics_all)
print(f"the number of deleted metrics: {del_num}")

# plt.figure(figsize=(25, 25))
# sns.heatmap(df_selected.corr(), vmax=1, vmin=-1, center=0, annot=True, square=True, cmap="bwr", annot_kws={"size": 16}, fmt=".2f", cbar_kws={"shrink": 0.85})
# plt.show()

['handler-write',
 'handler-commit',
 'cpu-usage_system',
 'handler-delete',
 'handler-read_rnd_next',
 'handler-external_lock',
 'handler-read_key',
 'handler-read_next',
 'handler-update',
 'avg_innodb_row_updates-inserted',
 'avg_innodb_row_reads-read',
 'avg_innodb_row_updates-updated',
 'avg_innodb_row_updates-deleted',
 'buffer_hit_ratio%',
 'disk_used_(%)',
 'innodb_io-mysql_global_status_innodb_data_written',
 'disk_used_(gb)-used',
 'innodb_io-mysql_global_status_innodb_data_read',
 'cpu-usage_iowait',
 'disk_busy',
 'network_io-bytes_sent',
 'avg_lock_wait_time',
 'cpu-usage_softirq',
 'handler-rollback',
 'handler-read_first',
 'disk_iops-writes',
 'handler-read_rnd',
 'network_io-bytes_recv',
 'cpu-usage_user',
 'handler-read_prev',
 'avg_mem_used']
the number of deleted metrics: 31


In [36]:
model = KMeans(n_clusters=2, random_state=100)

clusters = model.fit(df_selected)

# std_scaler = StandardScaler()
# df_std = std_scaler.fit_transform(df_selected)
# clusters = model.fit(df_std)

# mm_scaler = MinMaxScaler()
# df_normalized = mm_scaler.fit_transform(df_selected)
# clusters = model.fit(df_normalized)


In [37]:
for label_pred in clusters.labels_:
    print(label_pred, end=", ")
print('\n')
labels_int = labels.map({"read": 1, "write": 0}).to_numpy()
for label in labels_int:
    print(label, end=", ")

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,